<a href="https://colab.research.google.com/github/frikel12/Machine-Learning-and-Deep-Learningproject/blob/main/Fine_Tuning_BLOOMZ_LLM_for_QA_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers
!pip install accelerate==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.1
    Uninstalling huggingface-hub-0.19.1:
      Successfully uninstalled huggingface-hub-0.19.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
CSqa=load_dataset("commonsense_qa")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

In [ ]:
CSqa

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1221
    })
    test: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
        num_rows: 1140
    })
})

In [ ]:
import pandas as pd

common=pd.DataFrame(CSqa['validation'])

In [ ]:
common

,id,question,question_concept,choices,answerKey
0,1afa02df02c908a558b4036e80242fac,A revolving door is convenient for two directi...,revolving door,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
1,a7ab086045575bb497933726e4e6ad28,What do people aim to do at work?,people,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
2,b8c0a4703079cf661d7261a60a1bcbff,Where would you find magazines along side many...,magazines,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",B
3,e68fb2448fd74e402aae9982aa76e527,Where are you likely to find a hamburger?,hamburger,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
4,2435de612dd69f2012b9e40d6af4ce38,James was looking for a good place to buy farm...,farmland,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
...,...,...,...,...,...
1216,6c84e79d0595efd99596faa07c4961d0,What would you do to a rock when climb up a cl...,climb,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
1217,88f1fe6cfbcb1a25f25454341c789463,"His compressor needed a new hose, where did he...",hose,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",B
1218,5074bcaf0f700c9f3c8c563067af156a,"The man closed his eyes as the music played, w...",music,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",C
1219,6a253e076cd2af00e17d9950d70daf47,"Setting up framing, truss and beam are some of...",beam,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A


In [ ]:
common['choices'][0]

{'label': ['A', 'B', 'C', 'D', 'E'],
 'text': ['bank', 'library', 'department store', 'mall', 'new york']}

In [ ]:
def format_choices(choices):
    result = "\n".join(f"{label} {text}" for label, text in zip(choices['label'], choices['text']))
    return result
common['formatted_choices'] = common['choices'].apply(format_choices)


In [ ]:
common['formatted_choices'][4]

'A midwest\nB countryside\nC estate\nD farming areas\nE illinois'

In [ ]:
def buildprompt(data):
    prompt = {}
    prompt['text'] = "answer the following question by specifying the answer key: \n"+ data['question'] +"\n" + data['formatted_choices']+ "\nAnswer: " + data['answerKey']
    return prompt

common['prompt'] = common.apply(buildprompt, axis=1)


In [ ]:
common['prompt'][0]

{'text': 'answer the following question by specifying the answer key: \nA revolving door is convenient for two direction travel, but it also serves as a security measure at a what?\nA bank\nB library\nC department store\nD mall\nE new york\nAnswer: A'}

In [ ]:
import json
result = common['prompt'].to_list()
with open('prompts.json', 'w') as outfile:
    json.dump(result, outfile, ensure_ascii=False)

In [ ]:
dataset = load_dataset("json", data_files="prompts.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset['train']

Dataset({
    features: ['text'],
    num_rows: 1221
})

In [ ]:
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
import torch

checkpoint = "bigscience/bloomz-560m"

tokenizer = BloomTokenizerFast.from_pretrained(checkpoint)
model = BloomForCausalLM.from_pretrained(checkpoint).to("cuda")


In [ ]:
dataset['train']

Dataset({
    features: ['text'],
    num_rows: 1221
})

In [ ]:
def prepare_train_data(data):
    # prompt + completion
    text_input = data['text']
    # tokenize the input (prompt + completion) text
    tokenized_input = tokenizer(
      text_input,
      return_tensors='pt',
      padding='max_length',  # Pad to the maximum length
      max_length=93  # Set the maximum length
    )
    # generative models: labels are the same as the input
    tokenized_input['labels'] = tokenized_input['input_ids']
    return tokenized_input

train_dataset = dataset['train'].map(prepare_train_data,
                                     batched=True,
                                     remove_columns=["text"])

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1221
})

In [ ]:
training_arguments = TrainingArguments(
    'FT-bloom-560m',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=4,
    weight_decay=0.01,
    fp16=True,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset
)

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
500,2.056300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=608, training_loss=1.768100198946501, metrics={'train_runtime': 636.0845, 'train_samples_per_second': 7.678, 'train_steps_per_second': 0.956, 'total_flos': 820006549364736.0, 'train_loss': 1.768100198946501, 'epoch': 3.98})

In [ ]:
trainer.save_model()

In [ ]:
Ft_model = BloomForCausalLM.from_pretrained("/content/FT-bloom-560m").to('cuda')

In [ ]:
commontest=pd.DataFrame(CSqa['train']).head(100)

In [ ]:
commontest.head()

,id,question,question_concept,choices,answerKey
0,075e483d21c29a511267ef62bedc0461,The sanctions against the school were a punish...,punishing,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
1,61fe6e879ff18686d7552425a36344c8,Sammy wanted to go to where the people were. ...,people,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",B
2,4c1cb0e95b99f72d55c068ba0255c54d,To locate a choker not located in a jewelry bo...,choker,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
3,02e821a3e53cb320790950aab4489e85,Google Maps and other highway and street GPS s...,highway,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",D
4,23505889b94e880c3e89cff4ba119860,"The fox walked from the city into the forest, ...",fox,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",C


In [ ]:
commontest['formatted_choices'] = commontest['choices'].apply(format_choices)

In [ ]:
commontest['prompt'] = common.apply(buildprompt, axis=1)


In [ ]:
commontest['prompt']="answer the following question by specifying the answer key: \n"+ commontest['question'] +"\n" + commontest['formatted_choices']+ "\nAnswer: " + commontest['answerKey']

In [ ]:
commontest['prompt'][4]

'answer the following question by specifying the answer key: \nThe fox walked from the city into the forest, what was it looking for?\nA pretty flowers.\nB hen house\nC natural habitat\nD storybook\nE dense forest\nAnswer: C'

In [ ]:
#"finetuning-results"
results = []
for i, P in enumerate(commontest["prompt"]):
    prompt = P
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to('cuda:0') for k, v in inputs.items()}  # Move inputs to cuda:0

    # Ensure the model is on the same device (cuda:0)
    Ft_model.to('cuda:0')

    result = (tokenizer.decode(Ft_model.generate(inputs["input_ids"],
                       max_length=len(prompt) + 1,
                       do_sample=True,
                       top_k=50,
                       top_p=0.9
                      )[0]))
    results.append(result[len(prompt):])

In [ ]:
commontest['BloomZ-ft']=results


In [ ]:
commontest['BloomZ-ft'][2]

' jewelry store\nB safe deposit box\nC wallet\nD clerk\nAnswer: B safe deposit box\nC jewelery box\nE box\nAnswer: C jewelery box\nA box\nAnswer: D jewelry box\nAnswer: D boutique\nAnswer: E jewelry box\nA box\nAnswer: A jewelry box A small box can be found in a jewelry store where?\nA jewelery box\n Menjideen, where is it?\nB jewelery box\nC neck\nD boutique\nE jewelry box\nAnswer: A small box can be found in a jewelery shop where?\nA jewelery box\nB jewelry box\nE jewelery box\nA jewelry box A small box can be found in'

In [ ]:
#"Bloom-560m results"
#results1 = []
#for i, P in enumerate(commontest["prompt"]):
    #prompt = P
    #inputs = tokenizer(prompt, return_tensors="pt")
    #inputs = {k: v.to('cuda:0') for k, v in inputs.items()}  # Move inputs to cuda:0

    # Ensure the model is on the same device (cuda:0)
    #model.to('cuda:0')

    #result = (tokenizer.decode(model.generate(inputs["input_ids"],
                    #   max_length=len(prompt) + 1,
                     #  do_sample=True,
                      # top_k=50,
                      # top_p=0.9
                      #)[0]))
    #results.append(result[len(prompt)+1])

In [ ]:
#commontest['BloomZ-560m']=results

In [ ]:
commontest.to_csv('results.csv')